# Capstone - Battle of the Neighborhoods Week 2

### Applied Data Science Capstone - Identifying Food Deserts

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

Many low income areas across the US are without easy access to food of adequate nutritional quality (for example, a full-size grocery store such as Kroger's or HEB). Instead, these places may have nearby convenience stores or fast food restaurants, which exacerbate related health issues which often plague low income communities. This type of area is referred to as a "food desert." Most residents of food deserts also lack adequate transportation, presenting another barrier to overcoming the situation. 

In order to improve conditions within low-income communities, food supply and nutrition are one of the foundational areas Non-profit organizations will target. Some organizations open up their own grocery store, while others might look to solve for the transportation barrier. In this project, I will be looking to identify food deserts across the greater Houston area to both raise awareness and to highlight target areas for non-profit organizations looking to deploy resources.

Some of the criteria for this identification will be:
- Mean distance to grocery stores
- Population or density of residential addresses/apartment complexes
- Average income

## Data <a name="data"></a>

I will use Foursquare location data to identify neighborhoods with high distance to grocery stores, low distance to convenience stores and fast food, and potentially high density of residential apartment complexes. I will use an area of Galveston known as "North of Broadway" as a model, as this is a food desert which is known to me. I will look for other areas in Houston which match this dynamic.

I will also look to incorporate specific guidelines of distance and community constraints which are commonly cited:
- More than one mile from a grocery store
- Transportation constraints

## Analysis <a name="analysis"></a>

#### Import Packages

Download and Import Packages

In [3]:
#Uncomment any of the below lines to install geopy, folium and/or uszipcode if needed
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.5.0 --yes 
#!pip install uszipcode

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import uszipcode
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
search = SearchEngine()
print("Complete!")

print('Libraries imported.')

Complete!
Libraries imported.


#### Import Data from the Web

Import Houston Neighborhoods

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_Houston_neighborhoods"
neighborhoods = pd.read_html(url) [0]
neighborhoods.head()

,#,Name,Location relative to Downtown Houston,Approximate boundaries
0,1,Willowbrook,Northwest,Along Texas State Highway 249 northwest of Bel...
1,2,Greater Greenspoint,North,Around the junction of Beltway 8 and Interstat...
2,3,Carverdale,Northwest,South of the junction of Beltway 8 and U.S. Ro...
3,4,Fairbanks / Northwest Crossing,Northwest,Along U.S. Route 290 between Interstate 610 an...
4,5,Greater Inwood,Northwest,North of Fairbanks / Northwest Crossing and ea...


Import Houston Zip Codes

In [3]:
# There are 4 tables at this website. We will save the zip code table to a df named zip_codes
url = 'https://web.har.com/zipcode'
zip_codes = pd.read_html(url)[2]
zip_codes.head()

,0,1,2,3
0,Zip Code,City (City Alias Name(s)),County,NaN
1,77002,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",HARRIS,"77002 Details, Real Estate, Subdivisions and M..."
2,77003,"Houston - Inner Loop (HOUSTON,)",HARRIS,"77003 Details, Real Estate, Subdivisions and M..."
3,77004,"Houston - Inner Loop (HOUSTON,)",HARRIS,"77004 Details, Real Estate, Subdivisions and M..."
4,77005,"Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,...",HARRIS,"77005 Details, Real Estate, Subdivisions and M..."


In [4]:
# set the first row as the column headers
new_header = zip_codes.iloc[0] #grab the first row for the header
zip_codes = zip_codes[1:] #take the data less the header row
zip_codes.columns = new_header #set the header row as the df header
zip_codes.reset_index(drop=True, inplace=True) #reset index
zip_codes.head()

,Zip Code,City (City Alias Name(s)),County,NaN
0,77002,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",HARRIS,"77002 Details, Real Estate, Subdivisions and M..."
1,77003,"Houston - Inner Loop (HOUSTON,)",HARRIS,"77003 Details, Real Estate, Subdivisions and M..."
2,77004,"Houston - Inner Loop (HOUSTON,)",HARRIS,"77004 Details, Real Estate, Subdivisions and M..."
3,77005,"Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,...",HARRIS,"77005 Details, Real Estate, Subdivisions and M..."
4,77006,"Houston - Inner Loop (HOUSTON,)",HARRIS,"77006 Details, Real Estate, Subdivisions and M..."


In [5]:
# drop the unnecessary column (index location 3)
zip_codes.drop(zip_codes.columns[3], axis=1, inplace=True)
zip_codes.head()

,Zip Code,City (City Alias Name(s)),County
0,77002,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",HARRIS
1,77003,"Houston - Inner Loop (HOUSTON,)",HARRIS
2,77004,"Houston - Inner Loop (HOUSTON,)",HARRIS
3,77005,"Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,...",HARRIS
4,77006,"Houston - Inner Loop (HOUSTON,)",HARRIS


In [6]:
zip_codes.shape

(219, 3)

Get lat/long info for each zip code, using uszipcode

In [7]:
# demonstrate uszipcode search function, to find lat and long for a zip code
zipcode = search.by_zipcode(77573)
lat = zipcode.lat
lng = zipcode.lng
print(lat, lng)

29.5 -95.09


In [8]:
# while loop to iterate through each zip code and save to lists
lat_list=[]
lng_list=[] 
max_rows = zip_codes['Zip Code'].count()
i = 0
while i < max_rows:
    z = zip_codes.loc[i, 'Zip Code']
    zipcode = search.by_zipcode(z)
    lat = zipcode.lat
    lng = zipcode.lng
    lat_list.append(lat)
    lng_list.append(lng)
    i += 1

In [9]:
#add contents of lat and lng lists to our zip_codes dataframe
zip_codes['lat'] = lat_list
zip_codes['lng'] = lng_list
zip_codes.head()

,Zip Code,City (City Alias Name(s)),County,lat,lng
0,77002,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",HARRIS,29.75,-95.37
1,77003,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.75,-95.35
2,77004,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.72,-95.38
3,77005,"Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,...",HARRIS,29.72,-95.42
4,77006,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.74,-95.39


Drop rows where lat/long could not be located

In [10]:
zip_codes.dropna(inplace=True)
zip_codes.shape

(215, 5)

Mapping

Create a map of Houston using lat/long values

In [11]:
# create map of Houston using latitude and longitude values
latitude = zip_codes['lat'][0]
longitude = zip_codes['lng'][0]

houston_dots = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, zipcode, county in zip(zip_codes['lat'], zip_codes['lng'], zip_codes['Zip Code'], zip_codes['County']):
    label = '{}, {}'.format(zipcode, county)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(houston_dots)  
    
houston_dots

In [14]:
# create a map centered on Houston, using known lat/long
houston_map = folium.Map(location=[29.75, -95.37], tiles='OpenStreetMap', zoom_start=9)
houston_map

In [15]:
#import Houston zip code geojson from the web
geo_url = "https://opendata.arcgis.com/datasets/f392021d9d2344938b0958909d690cc7_0.geojson"

In [16]:
#map imported geojson for review
houston_map.choropleth(geo_data=geo_url)
houston_map

#### Pull venue data from Foursquare

Define Foursquare credentials and version

In [17]:
CLIENT_ID = 'Z32IMJE4TCTDQFAZQZLME1PQOMLGW3C4T4WVPFVKA4RKAQYD' # your Foursquare ID
CLIENT_SECRET = 'KGRTBFPBPBWC51ZIH1JVDSC0GHTJNUNIFFO5PMWGD4ITAA1I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z32IMJE4TCTDQFAZQZLME1PQOMLGW3C4T4WVPFVKA4RKAQYD
CLIENT_SECRET:KGRTBFPBPBWC51ZIH1JVDSC0GHTJNUNIFFO5PMWGD4ITAA1I


Explore first zip code in our zip_codes dataframe

In [18]:
input = zip_codes.loc[0, 'Zip Code']
input

'77002'

Set radius and limit for Foursquare query

In [19]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius

Create a function to pull venue information

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            "supermarket",
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            #v['venue']['location']['postalCode'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip Code', 
                  'Zip Code Latitude', 
                  'Zip Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                             #'Venue Zip Code',
                                               'Venue Category']
    
    return(nearby_venues)

Set Input Zip Code to Search

In [69]:
user_input = 77002
function_input = [user_input]
function_input

[77002]

Pull information for a given zip code

In [74]:
houston_venues = getNearbyVenues(names=function_input,
                                   latitudes=zip_codes['lat'],
                                   longitudes=zip_codes['lng']
                                  )

77002


In [71]:
houston_venues.head()

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,77002,29.75,-95.37,Phoenicia Specialty Foods,29.754502,-95.361760,Supermarket
1,77002,29.75,-95.37,Trader Joe's,29.739137,-95.411361,Grocery Store
2,77002,29.75,-95.37,Super H Mart,29.792324,-95.521111,Supermarket
3,77002,29.75,-95.37,H-E-B,29.747986,-95.485028,Supermarket
4,77002,29.75,-95.37,H Mart,29.706154,-95.553907,Supermarket


In [75]:
houston_venues.shape

(100, 7)

create full list of zip codes, lat and lng for each row in the zip_codes dataframe

In [43]:
zipcode_list = zip_codes['Zip Code']
lat_list = zip_codes['lat']
lng_list = zip_codes['lng']

Run the Foursquare query on the full list

In [44]:
#will need exception handler in the function, as some venues do not return postalcode data
#alternateively, we do not actually need to return the postalcode data to map the points, though we would want it for the choropleth map
#after running this, we will want to remove duplicate rows from the table, then map all points
temp_df = getNearbyVenues(names=zipcode_list,
                                   latitudes=lat_list,
                                   longitudes=lng_list
                                  )

77002
77003
77004
77005
77006
77007
77008
77009
77010
77011
77012
77013
77014
77015
77016
77017
77018
77019
77020
77021
77022
77023
77024
77025
77026
77027
77028
77029
77030
77031
77032
77033
77034
77035
77036
77037
77038
77039
77040
77041
77042
77043
77044
77045
77046
77047
77048
77049
77050
77051
77053
77054
77055
77056
77057
77058
77059
77060
77061
77062
77063
77064
77065
77066
77067
77068
77069
77070
77071
77072
77073
77074
77075
77076
77077
77078
77079
77080
77081
77082
77083
77084
77085
77086
77087
77088
77089
77090
77091
77092
77093
77094
77095
77096
77098
77099
77301
77302
77303
77304
77306
77316
77318
77327
77328
77336
77338
77339
77345
77346
77354
77355
77356
77357
77362
77365
77369
77372
77373
77375
77377
77378
77379
77380
77381
77382
77384
77385
77386
77388
77389
77396
77401
77406
77417
77422
77423
77429
77430
77433
77441
77444
77445
77446
77447
77449
77450
77459
77461
77464
77466
77469
77471
77476
77477
77478
77479
77480
77484
77486
77489
77493
77494
77498
77502
77503
7750

In [47]:
temp_df.head()

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,77002,29.75,-95.37,Phoenicia Specialty Foods,29.754502,-95.361760,Supermarket
1,77002,29.75,-95.37,Trader Joe's,29.739137,-95.411361,Grocery Store
2,77002,29.75,-95.37,H-E-B,29.737913,-95.402618,Grocery Store
3,77002,29.75,-95.37,Randalls,29.748830,-95.374711,Supermarket
4,77002,29.75,-95.37,Kroger,29.773624,-95.389976,Supermarket


In [48]:
temp_df.shape

(3125, 7)

In [50]:
houston_venues = temp_df
houston_venues.head()

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,77002,29.75,-95.37,Phoenicia Specialty Foods,29.754502,-95.361760,Supermarket
1,77002,29.75,-95.37,Trader Joe's,29.739137,-95.411361,Grocery Store
2,77002,29.75,-95.37,H-E-B,29.737913,-95.402618,Grocery Store
3,77002,29.75,-95.37,Randalls,29.748830,-95.374711,Supermarket
4,77002,29.75,-95.37,Kroger,29.773624,-95.389976,Supermarket


In [53]:
houston_venues.to_csv(r'C:\Users\cccap\OneDrive\Data Science\Courses\Capstone\houston_venues.csv', index=False)
print("df saved to csv")

df saved to csv


In [21]:
#uncomment the below to import csv data for expedited analysis
houston_venues = pd.read_csv('houston_venues.csv')
houston_venues.shape

(3125, 7)

Remove duplicate rows in the houston_venues dataframe

In [22]:
houston_venues.shape

(3125, 7)

In [23]:
houston_venues.drop_duplicates(subset=['Venue', 'Venue Latitude', 'Venue Longitude'], inplace=True)
houston_venues.shape

(325, 7)

In [24]:
houston_venues

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,77002,29.75,-95.37,Phoenicia Specialty Foods,29.754502,-95.361760,Supermarket
1,77002,29.75,-95.37,Trader Joe's,29.739137,-95.411361,Grocery Store
2,77002,29.75,-95.37,H-E-B,29.737913,-95.402618,Grocery Store
3,77002,29.75,-95.37,Randalls,29.748830,-95.374711,Supermarket
4,77002,29.75,-95.37,Kroger,29.773624,-95.389976,Supermarket
5,77002,29.75,-95.37,Kroger,29.754123,-95.405437,Supermarket
6,77002,29.75,-95.37,Kroger,29.727174,-95.430840,Supermarket
7,77002,29.75,-95.37,Kroger,29.742950,-95.392099,Supermarket
8,77002,29.75,-95.37,Whole Foods Market,29.749480,-95.461635,Grocery Store
9,77002,29.75,-95.37,Kroger,29.695869,-95.415516,Supermarket


Rename columns in the houston_venues dataframe

In [25]:
houston_venues.rename(columns={'Venue Latitude':'lat', 'Venue Longitude':'lng'}, inplace=True)
houston_venues.head()

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,lat,lng,Venue Category
0,77002,29.75,-95.37,Phoenicia Specialty Foods,29.754502,-95.361760,Supermarket
1,77002,29.75,-95.37,Trader Joe's,29.739137,-95.411361,Grocery Store
2,77002,29.75,-95.37,H-E-B,29.737913,-95.402618,Grocery Store
3,77002,29.75,-95.37,Randalls,29.748830,-95.374711,Supermarket
4,77002,29.75,-95.37,Kroger,29.773624,-95.389976,Supermarket


Map the venues

In [26]:
# create map of the identified venues
latitude = houston_venues['lat'][0]
longitude = houston_venues['lng'][0]

houston_venues_map = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, venue in zip(houston_venues['lat'], houston_venues['lng'], houston_venues['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(houston_venues_map)  
    
houston_venues_map

Get Census Data

In [27]:
#API call example for white 12-yr olds in Alabama (from US Census API site)
#api.census.gov/data/2010/dec/sf1?get=H001001,NAME&for=state:*&key=[user key]

## Results and DIscussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>